In [1]:
from IPython.display import HTML, display


display(HTML("<style>.container { width:" + f"{90}% " + "!important; }</style>"))


HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import numpy as np
import pandas as pd
from scipy import stats

import plotly.graph_objs as go
import plotly.express as px
import cufflinks as cf

import ipywidgets as widgets
from ipywidgets import interact

cf.go_offline()

In [3]:
df_hmean = pd.read_csv("data/h_mean.csv.gz", index_col=0, header=[0,1])
df_fft = pd.read_parquet("data/fft.parquet.gzip", engine="pyarrow")
str_locs = df_fft.columns

df_fft_roll = pd.read_csv("data/fft_roll.csv.gz").set_index(["freq", "timestamp"])

*Data here is only for 15 locations with biggest count of data points*

# Time series - Hourly mean pressure

In [4]:
grid_dict = dict(showgrid=True, gridcolor="black", gridwidth=0.1, dtick=2.5)

geo_dict = dict(projection_type="natural earth", showcoastlines=False, showrivers=True, showframe=False,
                showcountries=True, countrycolor="white",
                showland=True, landcolor="#c2c2c2",
                showocean=True, oceancolor="#e6fcfc",
                showlakes=True, lakecolor="#3399FF",
                lonaxis=grid_dict, lataxis=grid_dict)

scatter_geo_layout = go.Layout(geo=geo_dict)

In [5]:
@interact

def show_hmean(loc=str_locs):
    
    title = f"Hourly mean pressure for location: {loc}"
    fig = df_hmean[loc].iplot(subplots=True, shape=(2,1), title=title, legend=False, asFigure=True)
    
    fig.layout["yaxis"]["title"]["text"] = "Pressure [mb]"
    fig.layout["yaxis2"]["title"]["text"] = "Count"
    fig.show()
    
    
    tup_loc = loc.split(",")
    fig = px.scatter_geo(lat=[tup_loc[0]], lon=[tup_loc[1]], size=[20])
    fig.update_layout(scatter_geo_layout)
    fig.show()    

interactive(children=(Dropdown(description='loc', options=('32.5,-120.0', '32.5,-97.5', '37.5,-122.5', '37.5,-…

# PSD plot

In [6]:
# chi 2 rv value for given ci
rv = stats.chi2(df=2)
chi95 = rv.ppf(0.95)
chi99 = rv.ppf(0.99)
chi999 = rv.ppf(0.999)

@interact
def show_fft(loc=str_locs, logy=True):

    # main plot
    fig = df_fft[loc].dropna().iplot(asFigure=True, logy=logy, xTitle="Freq [cpd]", yTitle="Normalized PSD", 
                                          color="Navy", title=f"Location: {loc}")
    # add horizontal line
    fig.add_trace(go.Scatter(x=[0, 12], y=[chi95, chi95], name="95% ci",
                             mode="lines", line=dict(color="Red",dash="dashdot")))
    
    fig.add_trace(go.Scatter(x=[0, 12], y=[chi99, chi99], name="99% ci",
                             mode="lines", line=dict(color="Orange",dash="dashdot")))
    
    fig.add_trace(go.Scatter(x=[0, 12], y=[chi999, chi999], name="99.9% ci",
                             mode="lines", line=dict(color="Green",dash="dashdot")))
    
    # layout
    if logy:
        fig.update_layout(go.Layout(yaxis=dict(range=[-2,3])))
    
    
    fig.update_layout(showlegend=True)
    fig.show()

interactive(children=(Dropdown(description='loc', options=('32.5,-120.0', '32.5,-97.5', '37.5,-122.5', '37.5,-…

# PSD heat map - rolling FFT 

In [7]:
z_slider = widgets.FloatRangeSlider(value=(0,2), min=-8, max=4, step=0.5)

@interact

def show_fft_roll(loc=str_locs, colorscale=["purples", "spectral", "ylgnbu", "ylorrd"], 
                  inverted_colorscale=True, color_range=z_slider):
    
    hm = df_fft_roll[loc].unstack(0).dropna()
    
    if inverted_colorscale:
        cs = f"-{colorscale}"
    else:
        cs = colorscale
    
    np.log10(hm).iplot(kind="heatmap", yTitle="Frequency [cpd]", 
                       title=f"Log 10 Nomralized PSD for location: {loc}", colorscale=cs, 
                       zmin=color_range[0], zmax=color_range[1])

interactive(children=(Dropdown(description='loc', options=('32.5,-120.0', '32.5,-97.5', '37.5,-122.5', '37.5,-…